In [270]:
!pip install ipython-autotime
#%load_ext autotime

  Using cached https://files.pythonhosted.org/packages/59/0d/f5e65097c5b4847c36d2b4ad04995a04fc6b6c4c2587b052c7707a195ab0/ipython_autotime-0.1-py2-none-any.whl


Importing necessary libraries

In [271]:
import os
import time
import math 

from math import cos, acos, sqrt

Setting file directory and optimal solutions for each one

In [272]:
ds_dir = "tsp_dataset/"

In [273]:
data = [
    ["burma14.tsp", 3323],
    ["ulysses16.tsp", 6859],
    ["ulysses22.tsp", 7013],
    ["eil51.tsp", 426],
    ["berlin52.tsp", 7542],
    ["kroD100.tsp", 21294],
    ["kroA100.tsp", 21282],
    ["ch150.tsp", 6528],
    ["gr202.tsp", 40160],
    ["gr229.tsp", 134602],
    ["pcb442.tsp", 50778],
    ["d493.tsp", 35002],
    ["dsj1000.tsp", 18659688]
]

Defining a function to parse each file and gather the necessary data

In [274]:
def parser(file):
    
    lines = open(ds_dir + file, "r").readlines()
    index_start_coordinates = 0
    cont = 0
    V = []

    for line in lines:
      cont += 1
      if line.startswith("EOF") or line.startswith(" EOF"):
        break
      elif line.startswith("DIMENSION"):
        n = int(line.split(":")[1][1:])
      elif line.startswith("EDGE_WEIGHT_TYPE"):
        t = line.split(":")[1][1:-1] # TODO: remove space at the end
      elif line.startswith("NODE_COORD_SECTION"):
        index_start_coordinates = cont
      elif index_start_coordinates > 0:
        V.append((int(line.split()[0]) - 1, [float(line.split()[1]), float(line.split()[2])])) # (i, [x_value, y_value])
    #n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
    #t = lines[4].split()[1]

    return t, V

Heap data structure

In [275]:

class heap:
    def __init__(self):
        # desc: costruisco le strutture dati richieste dallo heap
        # A: coda di priorita, contiene la label di ciascuna 
        #    nodo ordinato nello heap secondo l'ordine di arrivo
        # position: dizionario che ha per chiave la label del nodo 
        #           e valore la sua posizione nella coda
        # value: dizionario che ha per chiave la label del nodo e 
        #        per valore il costo localmente migliore

        self.A = []
        self.position = dict()
        self.value = dict()

    def add(self,index,val):
        # desc: inserisce un nodo con label index nella coda ed il suo peso
        # index: label del nodo da inserire da inserire
        # value : costo di arrivo al nodo index
        # tempo : O(log(len(A)))

        self.value[index] = val
        n = len(self.A) 
        self.A.append(index)  #metto l'indice in ultima posizione
        self.position[index] = n
        self.bubbleUp(n)

    def isEmpty(self):
        # desc : True se la list e' vuota False altrimenti
        # tempo : O(1)

        return len(self.A) == 0
    
    def bubbleUp(self, i):
        # desc: aggiorna lo heap dopo l'inserimento o la modifica del nodo in posizione i
        # i : posizione del nodo da controllare
        # tempo : O(log(len(A)))

        p = (i-1)//2
        
        while i > 0 and self.value[self.A[i]] < self.value[self.A[p]]:
            #scambio A[i] <-> A[p]
            self.position[self.A[p]] = i
            self.position[self.A[i]] = p
            
            tmp = self.A[p]
            self.A[p] = self.A[i]
            self.A[i] = tmp
            
            i=p
            p=(i-1)//2

    def decreaseKey(self,node_index,newValue):
        # desc : decrementa la value e aggiorna la coda
        # node_index : label del nodo da aggiornare eventualmente
        # newValue : costo da confrontare con quello localmente migliore
        # tempo : O(log(len(A)))

        if self.value[node_index] < newValue:
            return False
        else: 
            self.value[node_index]=newValue
            i = self.position[node_index]
            self.bubbleUp(i)
            return True

    def extractMin(self):
        #desc: estrae il nodo con costo minimo e aggiorna la coda
        #tempo : O(log(len(A))
        
        n = len(self.A)

        minEl = self.A[0]
        del self.position[self.A[0]]
        del self.value[self.A[0]]
        
        self.A[0] = self.A[n-1]
        self.position[self.A[n-1]] = 0
        self.A.pop() #decremento n

        self.trickledown(0)
        return minEl

    def trickledown(self,i):
        #desc: aggiorna la posizione del nodo in posizione i 
        #      suoi figli hanno un costo minore 
        #i: posizione del nodo da controllare
        #tempo : O(log(len(A))

        l = i * 2 + 1
        r = i * 2 + 2 
        smallest = i
        if l < len(self.A) and self.value[self.A[l]] <  self.value[self.A[i]]:
            smallest = l

        if r < len(self.A) and self.value[self.A[r]] <  self.value[self.A[smallest]]:
            smallest = r
        
        if smallest != i:
            self.position[self.A[smallest]] = i
            self.position[self.A[i]] = smallest

            tmp = self.A[i]
            self.A[i]=self.A[smallest]
            self.A[smallest] = tmp
            self.trickledown(smallest)
    


GRAPH DEFINITION


In [276]:
def weight (u, v, t):
  if t == 'EUC_2D':
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)
    

In [277]:
class COMPLETE_GRAPH:

    def __init__(self,n):
        # desc : crea un grafo completo non orientato
        # n : numero di nodi del grafo
        # tempo : O(n^2)

        self.weights = [ [0 for i in range(n)] for j in range(n)]
        # matrice bidimensinale (lista di liste) preinit a 0

    def addWeight(self, x, y, weight):
        # desc : inserisce il peso weight tra i nodi con label x e y
        # x : label del primo nodo della coppia
        # y : label del secondo nodo della coppia
        # weight : peso da assegnare all'arco
        # tempo : O(1)

        if x == y: 
            self.weights[y][x] = 0
        else :
            self.weights[x][y] = weight
            self.weights[y][x] = weight
        
    
    def getWeight(self, i, j): 
        # desc : restituisce il peso dell'arco tra i e j
        # start_index : label del nodo da cui parte l'arco
        # dest_index : label del nodo a cui arriva l'arco
        # tempo : O(1)
        return self.weights[i][j]

    def getNumNodes(self):
        return len(self.weights)

    def createGraphFromTSP(fileName):
        # desc : carica i dati dal file fileName e crea un grafo che poi restituisce
        # fileName : nome del file da cui caricare i dati

        t, V = parser(fileName)
        G = COMPLETE_GRAPH(len(V))

        xy_pairs = [(x,y) for x in range(len(V)) for y in range(x + 1, len(V))]
        for (x,y) in xy_pairs:
            curr_weight = weight(V[x][1], V[y][1], t)
            G.addWeight(x, y, curr_weight)
            
        return G

    def weightCirc(self, circ):
        # desc : restituisce il costo totale se si attraversa il grafo secondo la sequenza data in circ
        # circ : circuito hamiltoniano 
        # tempo : O(len(circ)) 
        total_weight = 0
        for j in range(1, len(circ)):
            i = j-1
            dij = self.getWeight(circ[i],circ[j])
            total_weight += dij
        return total_weight


In [278]:
def PrimMST(G, root):
    #G grafo 
    #root : indice del vertice da cui fare partire l'albero

    n = G.getNumNodes()
    key, pi = keyPiInit(n,root,G)  

    #arco minimo che collega v all'albero parziale. 
    #None non c'e' arco per convenzione.

    #key[root] = 0 #non serva e' in keypi init
    Q = buildHeap(n, key, root)
    while Q.isEmpty() == False :
        u = Q.extractMin()
        #trovo l'arco minimo tra u e tutti gli altri nodi 
        for v in Q.A: #il grafo e' completo
            if G.getWeight(u,v) < key[v]:
                key[v] = G.getWeight(u,v)
                pi[v] = u
                Q.decreaseKey(v,key[v])
    
    return pi
 
def keyPiInit(dim,root,G):
    key = dict()
    pi = dict()
    for i in range(dim):
        key[i] = G.getWeight(root,i)
        pi[i] = root
    return key,pi


def buildHeap(n,keys,root):
    Q = heap()
    indexs = list(range(n))
    indexs.remove(root)
    for i in indexs: 
        Q.add(i,keys[i])
    return Q

def MST_TO_TREE(piMap):
    #desc: converte la mappa dei predecessori in un albero classico
    #piMap : mappa dei padri di ogni nodo
    #return : mappa dei successori
    if len(piMap) == 0 : return None
    else: 
        if len(piMap) == 1  :
            el,padre = piMap.popitem()
            return {padre: [el]}
        else:    
            el,padre = piMap.popitem()
            tree = pi2Tree(piMap)
            if padre in tree : tree[padre].append(el)
            else: tree[padre]=[el] 
            return tree

def costPi(piMap,G):
    somma = 0
    for p in piMap:
        el = piMap[p]
        somma += G.getweight(p,el)
    return somma


2 APPROX MST

In [279]:
def TWO_APPROX_TSP(G):
    # descr : algoritmo che risolve il problema TSP sul grafo G usando l'albero di copertura minimo di G. La soluzione ottenuta e' 2-approssimata
    # G : grafo pesato
    # ret : tupla contenente il circuito hamiltoniano approssima con il MST e il tempo di esecuzione dell'algoritmo

    start_time = time.time()
    
    MST = PrimMST(G, 0)
    TREE = MST_TO_TREE(MST)

    HAM_CYCLE = enumeration(TREE, 0)
    HAM_CYCLE.append(0)

    total_time = time.time() - start_time
    
    return HAM_CYCLE, total_time


In [280]:
def enumeration(tree, v):
    # desc : mette le label in una lista ordinata secondo una visita in profondita'
    # tree : mappa dei successori
    # v : vertice radice
    
    if v not in tree:
        return [v]
    
    A = [v]
    for c in tree[v] :
        if c != v : 
            A = A + enumeration(tree, c)

    return A


In [281]:
for file, optimal_solution in data:

    G = COMPLETE_GRAPH.createGraphFromTSP(file)

    sol, tm = MSTApprox(G)
    sol = G.weightCirc(sol)
    tm = '%.2E' % tm
    errore = round(float(sol-int(optimal_solution))/int(optimal_solution)*100,2)

    print("Filename: ", file)
    print("Optimal solution: ", optimal_solution)
    print("Costo soluzione: ", str(sol))
    print("Tempo di esecuzione: ", str(tm))
    print("Percentuale di errore: ", str(errore))    

    print()

Filename:  burma14.tsp
Optimal solution:  3323
Costo soluzione:  4003
Tempo di esecuzione:  3.18E-04
Percentuale di errore:  20.46

Filename:  ulysses16.tsp
Optimal solution:  6859
Costo soluzione:  7788
Tempo di esecuzione:  3.49E-04
Percentuale di errore:  13.54

Filename:  ulysses22.tsp
Optimal solution:  7013
Costo soluzione:  8308
Tempo di esecuzione:  5.77E-04
Percentuale di errore:  18.47

Filename:  eil51.tsp
Optimal solution:  426
Costo soluzione:  567
Tempo di esecuzione:  1.60E-03
Percentuale di errore:  33.1

Filename:  berlin52.tsp
Optimal solution:  7542
Costo soluzione:  10402
Tempo di esecuzione:  1.20E-03
Percentuale di errore:  37.92

Filename:  kroD100.tsp
Optimal solution:  21294
Costo soluzione:  28599
Tempo di esecuzione:  5.28E-03
Percentuale di errore:  34.31

Filename:  kroA100.tsp
Optimal solution:  21282
Costo soluzione:  30516
Tempo di esecuzione:  3.59E-03
Percentuale di errore:  43.39

Filename:  ch150.tsp
Optimal solution:  6528
Costo soluzione:  9315
Tem